## Compucell3D Project Week 5 Progress Report

### Goal:
- Fix bugs in widget
- Demo PottsCore and CellTypePlugin widgets with Sorting Demo (reference: Sorting Demo Notebook)
    - Add ContactPlugin parameters to widget
    - Add VolumePlugin widget
    - Add Initializer widget

### Progress:
- Fixed reset bug in PottsCore widget
- Fixed reset bug in CellTypePlugin widget
- Bug in PluginWidget where cell types were not being updated correctly

### Issues/Questions:
- Visualization incompatible with widget

In [1]:
from jupyter_widget_css import inject_jupyter_widget_css
inject_jupyter_widget_css()

## Parameters for the SortingDemo Simulation

---

## Summary Table

| Section         | What to Set                                    |
|-----------------|------------------------------------------------|
| Metadata        | (Simulation name, description, etc.)           |
| Potts Core      | Lattice size, neighbor order                   |
| Cell Types      | List of cell types                             |
| Volume Plugin   | Target volume and lambda for each cell type    |
| Contact Plugin  | Contact energies for all relevant type pairs   |
| Initializer     | Blob region(s) with geometry and cell types    |

### 1. **Metadata**
- (Usually includes simulation name, description, etc. — not explicitly shown in the demo, but typically present.)

---

### 2. **Potts Core**

| Parameter        | Value   | Description                                 |
|------------------|---------|---------------------------------------------|
| dim_x          | 100     | Lattice size in X                           |
| dim_y          | 100     | Lattice size in Y                           |
| neighbor_order | 2       | Neighborhood order (second-order neighbors) |

---

### 3. **Cell Types**

| Cell Type        |
|------------------|
| Condensing       |
| NonCondensing    |

---

### 4. **Volume Constraint (VolumePlugin)**

| Cell Type        | Target Volume | Lambda Volume |
|------------------|--------------|--------------|
| Condensing       | 25           | 2            |
| NonCondensing    | 25           | 2            |

---

### 5. **Contact (Adhesion) Energies (ContactPlugin)**

| Type 1        | Type 2        | Energy |
|---------------|---------------|--------|
| Medium        | Condensing    | 16     |
| Medium        | NonCondensing | 16     |
| Condensing    | Condensing    | 2      |
| Condensing    | NonCondensing | 11     |
| NonCondensing | NonCondensing | 16     |

---

### 6. **Initializer (BlobInitializer)**

| Region | Width | Radius | Center        | Cell Types                       |
|--------|-------|--------|---------------|----------------------------------|
| 1      | 5     | 20     | (50, 50, 0)   | Condensing, NonCondensing        |

In [2]:
from Spec_UI import SpecificationSetupUI

ui = SpecificationSetupUI()

In [4]:
print(ui.specs)

[<cc3d.core.PyCoreSpecs.Metadata object at 0x0000022334F86BF0>, <cc3d.core.PyCoreSpecs.PottsCore object at 0x0000022334F864D0>, <cc3d.core.PyCoreSpecs.CellTypePlugin object at 0x0000022334F86290>, <cc3d.core.PyCoreSpecs.VolumePlugin object at 0x0000022334F85B10>, <cc3d.core.PyCoreSpecs.ContactPlugin object at 0x0000022334F85C00>, <cc3d.core.PyCoreSpecs.BlobInitializer object at 0x0000022334F84FA0>]


In [3]:
from cc3d.core.PyCoreSpecs import BlobInitializer

specs = [
    ui.metadata,
    ui.potts_core,
    ui.cell_type_plugin
]

cell_types = [ui.cell_type_plugin.cell_types[1], ui.cell_type_plugin.cell_types[2]]

blob_init_specs = BlobInitializer()
blob_init_specs.region_new(width=5, radius=20, center=(50, 50, 0), cell_types=cell_types)
specs.append(blob_init_specs)

print(cell_types)

['Condensing', 'NonCondensing']


In [4]:

from cc3d.core.PyCoreSpecs import ContactPlugin

contact_specs = ContactPlugin(2)
contact_specs.param_new(type_1="Medium", type_2=cell_types[0], energy=16)
contact_specs.param_new(type_1="Medium", type_2=cell_types[1], energy=16)
contact_specs.param_new(type_1=cell_types[0], type_2=cell_types[0], energy=2)
contact_specs.param_new(type_1=cell_types[0], type_2=cell_types[1], energy=11)
contact_specs.param_new(type_1=cell_types[1], type_2=cell_types[1], energy=16)
specs.append(contact_specs)


In [8]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService
cc3d_sim = CC3DSimService()
cc3d_sim.register_specs(specs)
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()
vis_widget = cc3d_sim.visualize().show()
from IPython.display import display
display(vis_widget)
display(cc3d_sim.jupyter_run_button())

WILL RUN SIMULATION FROM BEGINNING


CC3DViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)

None

ToggleButton(value=False, description='Run Simulation')

In [11]:
# Testing vertical tabs
import ipywidgets as widgets
from IPython.display import display

# Create content for each tab
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

with out1:
    print("Content of Tab 1")

with out2:
    print("Content of Tab 2")

with out3:
    print("Content of Tab 3")

# Create individual buttons instead of ToggleButtons for more control
tab1_btn = widgets.Button(description='Tab 1', layout=widgets.Layout(width='100%'))
tab2_btn = widgets.Button(description='Tab 2', layout=widgets.Layout(width='100%'))
tab3_btn = widgets.Button(description='Tab 3', layout=widgets.Layout(width='100%'))

# Put them into a VBox so they stack tightly
tabs_box = widgets.VBox([tab1_btn, tab2_btn, tab3_btn], layout=widgets.Layout(width='120px'))

# Output area
content_area = widgets.Output()

# Function to switch content
def show_tab(tab):
    content_area.clear_output()
    with content_area:
        if tab == 1:
            display(out1)
        elif tab == 2:
            display(out2)
        elif tab == 3:
            display(out3)

# Attach callbacks
tab1_btn.on_click(lambda b: show_tab(1))
tab2_btn.on_click(lambda b: show_tab(2))
tab3_btn.on_click(lambda b: show_tab(3))

# Layout the tabs and content
ui = widgets.HBox([tabs_box, content_area])

# Display
display(ui)

# Initialize first tab
show_tab(1)


In [12]:
import ipywidgets as widgets
from IPython.display import display
from cc3d.core.PyCoreSpecs import PLUGINS

for plugin_cls in PLUGINS:
    plugin = plugin_cls()
    print(f"Plugin: {plugin_cls.__name__}")
    for param_name in plugin.spec_dict.keys():
        # For demonstration, create a text box for each parameter
        value = plugin.spec_dict[param_name]
        input_widget = widgets.Text(
            value=str(value),
            description=param_name
        )
        display(input_widget)

Plugin: AdhesionFlexPlugin


Text(value='1', description='neighbor_order')

Text(value='[]', description='molecules')

Text(value='{}', description='densities')

Text(value='{}', description='binding_formulas')

...

Text(value='{}', description='field_specs')

Plugin: SurfacePlugin


Text(value='{}', description='params')

Plugin: VolumePlugin


Text(value='{}', description='params')

In [13]:
from cc3d.core.PyCoreSpecs import PLUGINS
print([p.__name__ for p in PLUGINS])

['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'CellTypePlugin', 'CenterOfMassPlugin', 'ChemotaxisPlugin', 'ConnectivityGlobalPlugin', 'ConnectivityPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'MomentOfInertiaPlugin', 'NeighborTrackerPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'SurfacePlugin', 'VolumePlugin']
